# TwitterAPIと形態素解析でフォロワーのいいねしたツイートの傾向を分析するレシピ

#### 初期化

In [109]:
# 初期化
import time
from collections import Counter
from io import StringIO

import MeCab
import pandas as pd
import tweepy

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

import re

from IPython.display import HTML

## まずは資料通り実行

### OAuth認証

In [69]:
CONSUMER_KEY = "0RQGlWHRlYK1D2sPmgHMr72jl"
CONSUMER_SECRET = "RYCcL5XWUpWpfNmycwXXFsGaGCp3jWBscJYXT3m6T2VPlAC9cR"
ACCESS_TOKEN = "1550863859220889600-L95YqiUqxtNDvU6uW2A13fXoeJnRQH"
ACCESS_SECRET = "43UivRhHsh72RL8FO0VUwreKtx6fCOAV6h21DDFbA27Rf"

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
# OAuth認証を行うためのものです。
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
# アクセストークンをセットしておきます
api = tweepy.API(auth)
# apiオブジェクトにAPIを入れておきます。

### TwitterAPIから取得

In [ ]:
search_results = api.get_follower_ids(screen_name="ynu_bbc", count=50)
# レシピは自分のフォロワーのいいねの傾向を分析しますが、今回は一時的なアカウントでフォロワーがいないため横国生協のフォロワー≒横国生or横国OB.OGのいいねの傾向を分析しようと思う。
# フォロワーのID情報を取得しています。（count=50は50件取得の意）
# レシピにはapi.followers_idsとありますがgetを挿入してください。
favorite_tweets = ""

for result in search_results:
    try:
        time.sleep(1)
        # 1秒処理を待たせるものですが、これはなんで入ってるのかわかりません。なくても動きました。
        # あった方が10秒ほど早く動くかもしれません。
        # 理由は以下の通りかもしれません。
        # 読み込みに時間がかかる処理を実行した際、読み込み完了前にアクセスすると正しく読み込めずエラーになる場合があります。読み込み完了までの時間の猶予を生むために利用できます。
        response_list = api.get_favorites(user_id=result)
        # 同じくgetを挿入
        # 先ほど取得したIDのユーザーのいいねを押したツイートを所得し、オブジェクトに入れます。
        for response in response_list:
            favorite_tweets += response._json["text"]
            # jsonとは｢JavaScript Object Notation｣で、JavaScriptのオブジェクト記法を用いたデータ交換フォーマット」です。Jsonについての詳細はリンクを張ったので各自参照。
            # 取得したツイートデータにはツイート本文以外の情報も含まれているので（Json形式）、_json['text']とすることで本文のみを取り出します。textを指定しないとエラーが起きます。
            # エラーコード（can only concatenate str (not "dict") to str）←これはよくわからなかったです。
    except Exception as e:
        print(e)

print(favorite_tweets)

# try,except構文は、例外が発生し、処理が止まるのを防ぐため例外を変数eに格納するためのコード。

### ipadicでの実行

In [38]:
with open("ynu_bbc_text.txt", "r", encoding="UTF-8") as f:
    text = f.read()

# print(text.replace('\n', '').replace(' ', ''))
tagger = MeCab.Tagger()
buffer = StringIO(tagger.parse(text.replace("\n", "").replace(" ", "")))

df = pd.read_csv(
    buffer,
    names=["表層形", "発音", "読み", "原形", "品詞", "品詞再分類1", "品詞細分類2", "品詞細分類3", "活用型", "活用形"],
    skipfooter=1,
    sep="[\t]",
    engine="python",
)

# display(df.count())
display(df.head())
noun_df = df[df["品詞"].str.contains("名詞") & ~df["品詞"].str.contains("代名詞", na=False)]

c = Counter(noun_df["表層形"])
print(c.most_common(30))

表層形    発音    読み      原形          品詞 品詞再分類1 品詞細分類2 品詞細分類3  活用型  活用形
0   【   NaN   NaN       【    補助記号-括弧開   None   None   None  NaN  NaN
1   北    キタ    キタ       北  名詞-普通名詞-一般    NaN    NaN    0,2  NaN  NaN
2  ガス    ガス    ガス  ガス-gas  名詞-普通名詞-一般    NaN    NaN      1  NaN  NaN
3  硬式  コーシキ  コウシキ      硬式  名詞-普通名詞-一般    NaN    NaN      0  NaN  NaN
4  野球  ヤキュー  ヤキュウ      野球  名詞-普通名詞-一般    NaN    NaN      0  NaN  NaN

[('t', 276), ('co', 273), ('https', 245), ('1', 81), ('野球', 73), ('7', 65), ('4', 63), ('3', 62), ('2', 61), ('5', 60), ('日', 47), ('6', 44), ('0', 43), ('戦', 42), ('年', 42), ('選手', 41), ('8', 41), ('高校', 39), ('回', 37), ('大会', 37), ('日本', 37), ('大学', 36), ('人', 36), ('部', 35), ('9', 35), ('試合', 33), ('さん', 30), ('こと', 29), ('今日', 22), ('チーム', 18)]


### mecab-ipadic-neologdでの実行

In [37]:
with open("ynu_bbc_text.txt", "r", encoding="UTF-8") as f:
    text = f.read()

# print(text.replace('\n', '').replace(' ', ''))
tagger = MeCab.Tagger(r'-d "/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"')
buffer = StringIO(tagger.parse(text.replace("\n", "").replace(" ", "")))

df = pd.read_csv(
    buffer,
    names=["表層形", "品詞", "品詞細分類1", "品詞細分類2", "品詞細分類3", "活用型", "活用形", "原形", "読み", "発音"],
    skipfooter=1,
    sep="[\t,,]",
    engine="python",
)

# display(df.count())
display(df.head())
noun_df = df[df["品詞"].str.contains("名詞") & ~df["品詞"].str.contains("代名詞", na=False)]

c = Counter(noun_df["表層形"])
print(c.most_common(30))

表層形  品詞 品詞細分類1 品詞細分類2 品詞細分類3 活用型 活用形      原形         読み         発音
0      【  記号    括弧開      *      *   *   *       【          【          【
1    北ガス  名詞   固有名詞     一般      *   *   *     北ガス       キタガス       キタガス
2  硬式野球部  名詞   固有名詞     一般      *   *   *   硬式野球部  コウシキヤキュウブ  コーシキヤキューブ
3      ⚾  名詞     一般      *      *   *   *  ベースボール     ベースボール     ベースボール
4      】  記号    括弧閉      *      *   *   *       】          】          】

[('t', 273), ('co', 273), ('https', 246), ('7', 49), ('1', 46), ('5', 44), ('の', 37), ('0', 35), ('8', 32), ('6', 30), ('ん', 30), ('こと', 29), ('4', 24), ('2', 23), ('3', 23), ('9', 23), ('試合', 22), ('選手', 22), ('今日', 22), ('さん', 21), ('よう', 20), ('高校野球', 17), ('人', 16), ('部', 16), ('応援', 15), ('私', 15), ('日', 15), ('大会', 14), ('これ', 14), ('本日', 13)]


## 自分のアカウントで再実行

### OAuth認証

In [93]:
# 環境変数の設定
CONSUMER_KEY = "0RQGlWHRlYK1D2sPmgHMr72jl"
CONSUMER_SECRET = "RYCcL5XWUpWpfNmycwXXFsGaGCp3jWBscJYXT3m6T2VPlAC9cR"
ACCESS_TOKEN = "1550863859220889600-L95YqiUqxtNDvU6uW2A13fXoeJnRQH"
ACCESS_SECRET = "43UivRhHsh72RL8FO0VUwreKtx6fCOAV6h21DDFbA27Rf"

# 自分がOAuthを使ってアクセスするAPIがどれか指定します
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)

# アクセストークンを使って、そこにアクセスする権限があることを証明します
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

# 認証情報(auth)をtweepy.APIに渡して、接続済みの各種処理が可能な大元（おおもと）の変数を作成します
api = tweepy.API(auth)

ids [79765255, 1368550906036252676, 1339974340499046402, 1498841626705534977, 1527247041524400128]
401 Unauthorized
Not authorized.


### TwitterAPIを使って取得と整形・保存

In [101]:
# まずは元となるアカウントを指定してそのフォロワーを取得します。
# screen_nameに元にしたいtwitterのid名を入れ、countに取得したいユーザーのidの数を指定します。
target_id = "dominisikokura"
time.sleep(1)
ids = api.get_follower_ids(screen_name=target_id, count=5)
favorite_tweets = ""
for id in ids:
    try:
        # 取得時にアクセスを集中させすぎてDDos攻撃にならないようにするためにアクセスに間隔を空けます。
        time.sleep(1)
        response_list = api.get_favorites(user_id=id)
        for response in response_list:
            print(response._json["text"])
            print("----------------------")
            favorite_tweets += response._json["text"]
    except Exception as e:
        print(e)

# 取得したツイート群をファイルに書き込んで保存する処理

# 空のファイルを書き込みモードで開いて
f = open(target_id + ".txt", "w")

# 書き込んで
f.write(favorite_tweets)

# 閉じる
f.close()

ファンダムの課題とな＞Billboard JAPAN運営が警鐘「チャート操作目的の再生では、音楽を“聴く”とは言えない」 https://t.co/9jlO8sAbnY
----------------------
米国初！Cruise、自動運転タクシーを海外に「輸出」へ https://t.co/coaFwQA3fG @jidountenlabから
----------------------
web3のブームの一番のポイントは、中央集権か、非中央集権か、という権力の話よりも、どこまで情報がオープンになるか、オープンとクローズドのどちらが進化が早いかの比較なんじゃないかと思ってきました。
----------------------
分かる。俺も2社目のJTCに転職したら攻撃的になっていってしまったな。物静かな感じでいると周りから付け込まれて仕事を押し付けられたり、ミスを自分のせいにされたりするので、自分の身を守るために攻撃的にならざるを得ないのよね。
----------------------
早朝アンパンマン映画、終了10分前で限界が来て退出。去年よりは見ていられたので成長。

シアターに我が家含めて2組。
ONE PIECEでお忙しいだろうに上映してくれた映画館に感謝。
----------------------
採用面接でどう遠慮せずに切り込むか、についてまとめてみました。https://t.co/Mj4iINFjvG
----------------------
電力会社も動き始めていますね　東京電力EP、法人顧客のカーボンニュートラル実現を支えるカーボンニュートラルソリューションサービスを本格開始　https://t.co/QWCz4KhVlz #脱炭素　#再エネ　#電力自由化 https://t.co/wj8y0dHINU
----------------------
東京ラブストーリーのカンチはクソ男でリカは最高にいい女だという事が発覚した。
----------------------
8/26,27,28にマネーフォワードさんと金融教育のイベント＆ワークショップを開催します。ぜひご応募ください。（自分はこれから27日の資料づくりをがんばります）
https://t.co/kai4XCq4Yy
----------------------
ご

### ipadicで実行

In [113]:
with open("dominisikokura.txt", "r", encoding="UTF-8") as f:
    text = f.read()
    text = text.replace("\r\n", "").replace("\n", "").replace(" ", "")

tagger = MeCab.Tagger()
buffer = StringIO(tagger.parse(text))

df = pd.read_csv(
    buffer,
    names=["表層形", "発音", "読み", "原形", "品詞", "品詞再分類1", "品詞細分類2", "品詞細分類3", "活用型", "活用形"],
    skipfooter=1,
    sep="[\t]",
    engine="python",
)

# display(df.count())
display(df.head())
noun_df = df[df["品詞"].str.contains("名詞") & ~df["品詞"].str.contains("代名詞", na=False)]

c = Counter(noun_df["表層形"])
df = pd.DataFrame(c.most_common(30), columns=["単語", "回数"])

display(HTML("<h1>Twitter フォロワー分析</h1>"))
display(df)

表層形    発音    読み   原形             品詞 品詞再分類1 品詞細分類2 品詞細分類3  活用型  活用形
0  にゃん   ニャン   ニャン  にゃん             副詞    NaN    NaN      1  NaN  NaN
1    。   NaN   NaN    。        補助記号-句点   None   None   None  NaN  NaN
2   世界   セカイ   セカイ   世界     名詞-普通名詞-一般    NaN    NaN    1,2  NaN  NaN
3  不思議   フシギ   フシギ  不思議  名詞-普通名詞-形状詞可能    NaN    NaN      0  NaN  NaN
4   発見  ハッケン  ハッケン   発見   名詞-普通名詞-サ変可能    NaN    NaN      0  NaN  NaN

単語   回数
0       t  272
1      co  270
2   https  245
3       1   72
4       2   69
5       人   66
6       3   56
7      こと   55
8       4   45
9       8   43
10      横   42
11      6   41
12      方   39
13     さん   39
14      国   39
15      7   39
16     大学   38
17      9   35
18      5   35
19     参加   34
20      春   34
21      0   33
22      笑   33
23      会   30
24     時間   28
25  エンジニア   26
26      回   25
27      日   25
28     自分   23
29      者   23

### mecab-ipadic-neologdでの実行

In [112]:
tagger = MeCab.Tagger(r'-d "/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"')
buffer = StringIO(tagger.parse(text))

df = pd.read_csv(
    buffer,
    names=["表層形", "品詞", "品詞細分類1", "品詞細分類2", "品詞細分類3", "活用型", "活用形", "原形", "読み", "発音"],
    skipfooter=1,
    sep="[\t,,]",
    engine="python",
)

display(df.head())
noun_df = df[df["品詞"].str.contains("名詞") & ~df["品詞"].str.contains("代名詞", na=False)]

c = Counter(noun_df["表層形"])
df = pd.DataFrame(c.most_common(30), columns=["単語", "回数"])

display(HTML("<h1>Twitter フォロワー分析</h1>"))
display(df)

表層形   品詞 品詞細分類1 品詞細分類2 品詞細分類3     活用型  活用形       原形          読み  \
0      にゃん   副詞     一般      *      *       *    *      にゃん         ニャン   
1        。   記号     句点      *      *       *    *        。           。   
2  世界不思議発見   名詞   固有名詞     一般      *       *    *  世界不思議発見  セカイフシギハッケン   
3        、   記号     読点      *      *       *    *        、           、   
4       いい  形容詞     自立      *      *  形容詞・イイ  基本形       いい          イイ   

           発音  
0         ニャン  
1           。  
2  セカイフシギハッケン  
3           、  
4          イイ

単語   回数
0       t  270
1      co  270
2   https  266
3       の   64
4      こと   55
5       人   48
6      横国   42
7       8   37
8       方   35
9       春   35
10     よう   33
11      7   32
12      2   31
13      6   31
14     参加   30
15      3   29
16      0   27
17      1   26
18      5   26
19  エンジニア   26
20      9   23
21     自分   19
22     大学   19
23     今日   18
24   動画編集   18
25     さん   17
26    説明会   17
27      ん   17
28      笑   17
29     仕事   15

## 課題点の列挙

In [68]:
# いくつか問題点が見つかる

# 1. https://t.co/が多く含まれている
# 下記は「：」や「/」が名詞分類に混ざっていないことの確認のための出力(空出力が望ましい)
display(
    noun_df[
        noun_df["表層形"].str.contains(":", na=False)
        | noun_df["表層形"].str.contains("/", na=False)
    ]
)


# 2. https://t.co/ランダム英数字のランダム英数字の部分が直後の文字列と結合して正しく判別されていない(下記は一例)
display(noun_df[noun_df["表層形"].str.contains("iPhone")])

Empty DataFrame
Columns: [表層形, 品詞, 品詞細分類1, 品詞細分類2, 品詞細分類3, 活用型, 活用形, 原形, 読み, 発音]
Index: []

表層形  品詞 品詞細分類1 品詞細分類2 品詞細分類3 活用型 活用形      原形     読み     発音
1120  AwUgfiPhone  名詞     一般      *      *   *   *       *   None   None
1168       iPhone  名詞   固有名詞     一般      *   *   *  iPhone  アイフォン  アイフォン

## 課題点を改善

In [125]:
target_id = "dominisikokura"
time.sleep(1)
ids = api.get_follower_ids(screen_name=target_id, count=50)
favorite_tweets = ""
for id in ids:
    try:
        time.sleep(1)
        response_list = api.get_favorites(user_id=id)
        for response in response_list:
            fixed_tweet = re.sub(
                r"https.*$", "", response._json["text"], flags=re.MULTILINE
            )
            favorite_tweets += fixed_tweet
            print(fixed_tweet)
            print("----------------------")
    except Exception as e:
        print(e)


f = open(target_id + ".txt", "w")
f.write(favorite_tweets)
f.close()

ファンダムの課題とな＞Billboard JAPAN運営が警鐘「チャート操作目的の再生では、音楽を“聴く”とは言えない」 
----------------------
米国初！Cruise、自動運転タクシーを海外に「輸出」へ 
----------------------
web3のブームの一番のポイントは、中央集権か、非中央集権か、という権力の話よりも、どこまで情報がオープンになるか、オープンとクローズドのどちらが進化が早いかの比較なんじゃないかと思ってきました。
----------------------
分かる。俺も2社目のJTCに転職したら攻撃的になっていってしまったな。物静かな感じでいると周りから付け込まれて仕事を押し付けられたり、ミスを自分のせいにされたりするので、自分の身を守るために攻撃的にならざるを得ないのよね。
----------------------
早朝アンパンマン映画、終了10分前で限界が来て退出。去年よりは見ていられたので成長。

シアターに我が家含めて2組。
ONE PIECEでお忙しいだろうに上映してくれた映画館に感謝。
----------------------
採用面接でどう遠慮せずに切り込むか、についてまとめてみました。
----------------------
電力会社も動き始めていますね　東京電力EP、法人顧客のカーボンニュートラル実現を支えるカーボンニュートラルソリューションサービスを本格開始　
----------------------
東京ラブストーリーのカンチはクソ男でリカは最高にいい女だという事が発覚した。
----------------------
8/26,27,28にマネーフォワードさんと金融教育のイベント＆ワークショップを開催します。ぜひご応募ください。（自分はこれから27日の資料づくりをがんばります）

----------------------
ご存じ星野リゾート星野さんと代表池田の対談記事です。 学び多い、そして自分のことのように嬉しい

----------------------
キース・ジャレット聴きながら昼寝。良き
----------------------
耕地放棄地の活用、地元雇用の創出、持続可能な調達、地方創生などへコーヒーでアプローチ！
--------------

## 課題点改善後再実行

In [126]:
with open("dominisikokura.txt", "r", encoding="UTF-8") as f:
    text = f.read()
    text = text.replace("\r\n", "").replace("\n", "").replace(" ", "")
tagger = MeCab.Tagger(r'-d "/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"')
buffer = StringIO(tagger.parse(text))

df = pd.read_csv(
    buffer,
    names=["表層形", "品詞", "品詞細分類1", "品詞細分類2", "品詞細分類3", "活用型", "活用形", "原形", "読み", "発音"],
    skipfooter=1,
    sep="[\t,,]",
    engine="python",
)

display(df.head())
noun_df = df[df["品詞"].str.contains("名詞") & ~df["品詞"].str.contains("代名詞", na=False)]

c = Counter(noun_df["表層形"])
df = pd.DataFrame(c.most_common(30), columns=["単語", "回数"])

display(HTML("<h1>Twitter フォロワー分析</h1>"))
display(df)

表層形  品詞 品詞細分類1 品詞細分類2 品詞細分類3 活用型 活用形     原形     読み     発音
0  ファンダム  名詞   固有名詞     一般      *   *   *  ファンダム  ファンダム  ファンダム
1      の  助詞    連体化      *      *   *   *      の      ノ      ノ
2     課題  名詞     一般      *      *   *   *     課題    カダイ    カダイ
3      と  助詞    格助詞     引用      *   *   *      と      ト      ト
4      な  助詞    終助詞      *      *   *   *      な      ナ      ナ

単語  回数
0           横国  24
1           こと  23
2            の  23
3           よう  21
4            人  21
5            春  18
6            8  16
7        エンジニア  16
8            笑  14
9         サークル  14
10           方  13
11          さん  12
12  Scaniverse  12
13          新歓  12
14          相談  12
15          自分  11
16          参加  11
17        サービス  11
18     mentory  11
19          月額  11
20         プラン  11
21          柔軟  11
22          今日  10
23        2022   9
24         募集中   8
25          大学   8
26          大根   8
27          特集   7
28          東京   7
29          出張   7